## Project Applied Statistics - Winter 2024 (40%)
Lecturer: Ian McLoughlin
Author: Lais Coletta Pereira

Your task is to perform t-tests and ANOVA on this dataset while describing the dataset and explaining your work. In doing this you should:

1. Download and save the dataset to your repository.

2. Describe the data set in your notebook.

3. Describe what a t-test is, how it works, and what the assumptions are.

4. Perform a t-test to determine whether there is a significant difference between the two treatment groups trt1 and trt2.

5. Perform ANOVA to determine whether there is a significant difference between the three treatment groups ctrl, trt1, and trt2.

6. Explain why it is more appropriate to apply ANOVA rather than several t-tests when analyzing more than two groups.

Marking Scheme
Each component will be graded based on the following four categories. Each category carries equal weight.

Remember, your repository is what will be evaluated. It should demonstrate evidence of the criteria outlined for each category. That said, the examiners' overall impression of the submission may affect marks in each category.

You are expected to make steady progress on the assessment throughout the semester. This should be reflected in your commit history. Huge commits, especially late in the semester, will not be accepted. At any stage you may be asked to discuss the work to date in your repository.

If you encounter issues with your repository, seek help well before the deadline. Do not delete any files or commits without first consulting the lecturer.

Research
Evidence of research on relevant topics.
Appropriate referencing.
Building upon the literature and documentation.
Comparisons to similar work.
Development
Clear, concise, and correct code.
Appropriate tests.
Knowledge of different approaches and algorithms.
Clean architecture.
Documentation
Clear explanations of concepts.
Concise comments in code and elsewhere.
Appropriate README for repository.
Consistency
Tens of commits, each representing a reasonable amount of work.
Literature, documentation, and code evidencing work on the assessment.
Evidence of reviewing and refactoring.
Advice
The freedom provided in open-style assessments such as this one can feel challenging. You need to decide how to start, what content to include, how much to cover, and how to make the work your own.

The assessment is designed to provide you with opportunities for independent thinking and decision-making. Employers value graduates who can take initiative, work independently, and make design decisions with minimal guidance. We expect you to have basic programming knowledge and be able to find information on your own.

You should have a clear plan before you start coding. Your submission should include both your plan and an explanation of any design choices you made.

Make sure to follow ATU's policies and regulations which are on the Student Hub. Pay particular attention to any policies on plagiarism and the Student Code. If you're unsure about anything, ask the lecturer.

1. Download and Load the Dataset

2. Describe the Dataset

3. What is a t-test?
A t-test is a statistical test used to determine whether there is a significant difference between the means of two groups. There are two main types of t-tests:

Independent samples t-test: Compares means of two independent groups.
Paired samples t-test: Compares means from the same group at two different times.
How It Works
Computes the t-statistic, which measures the difference between group means relative to the variation in the data.
Compares the t-statistic to a critical value from the t-distribution to decide whether the difference is statistically significant.
Assumptions
The data is continuous.
The data follows a normal distribution (or the sample size is large enough for the Central Limit Theorem to apply).
The two groups have approximately equal variances.
Observations are independent.
4. Performing t-test on trt1 and trt2
We will check if there is a significant difference in plant weights between trt1 and trt2.

5. What is ANOVA?
Analysis of Variance (ANOVA) is used to determine whether there are statistically significant differences between the means of three or more groups.

How It Works
Partition total variation in the data into "between-group" and "within-group" variances.
Calculates the F-statistic, which is the ratio of between-group variance to within-group variance.
Compares the F-statistic to a critical value from the F-distribution.
Assumptions
The data follows a normal distribution.
Homogeneity of variances across groups.
Observations are independent.
6. Why Use ANOVA Instead of Multiple t-tests?
Increased Error Rate: Conducting multiple t-tests increases the likelihood of Type I errors (false positives) because each test has its own significance threshold.
Holistic Approach: ANOVA considers all group comparisons simultaneously, providing a single test statistic.
Efficiency: ANOVA is computationally more efficient and interprets overall group differences in one step.
7. Performing ANOVA
We will determine if there is a significant difference in plant weights between the three groups: ctrl, trt1, and trt2.

Import Libraries

In [12]:
# Import libraries
import pandas as pd
import ssl


_1. Download and Load the Dataset_

In [13]:
import pandas as pd

# URL of the dataset
url = "https://vincentarelbundock.github.io/Rdatasets/csv/datasets/PlantGrowth.csv"

# Load the dataset from the URL
df = pd.read_csv(url)

# Define the path to save the dataset inside your 'Applied-Statistics' folder
save_path = "Applied-Statistics/plantgrowth.csv"

# Save the dataset to the specified path
df.to_csv(save_path, index=False)

print(f"Dataset successfully saved to {save_path}")

OSError: Cannot save file into a non-existent directory: 'Applied-Statistics'

In [14]:
# Disable SSL verification (for development purposes only)
ssl._create_default_https_context = ssl._create_unverified_context

# URL of the dataset
url = "https://vincentarelbundock.github.io/Rdatasets/csv/datasets/PlantGrowth.csv"

# Load the dataset from the URL
df = pd.read_csv(url)

# Define the path to save the dataset inside the 'Applied-Statistics' folder
save_path = "/Desktop/DA Final Semester/Applied-Statistics/plantgrowth.csv"

# Save the dataset to the specified path
df.to_csv(save_path, index=False)

print(f"Dataset successfully saved to {save_path}")


OSError: Cannot save file into a non-existent directory: '/Desktop/DA Final Semester/Applied-Statistics'

2. Describe the data set in your notebook:

print(plant_growth.describe())

Observations: 
1. The weight variable shows moderate variability (std = 0.701), with weights ranging from 3.59 to 6.31 grams.
2. The median weight (5.155) is close to the mean (5.073), indicating a symmetric distribution.

In [ ]:
# More detailed dataset description
# Dataset overview
print("\nDataset Information:")
plant_growth.info()  # This automatically prints out column types and non-null counts

# Check for missing values and report if any
missing_values = plant_growth.isnull().sum()
print("\nMissing Values in Dataset:")
print(missing_values if missing_values.any() else "No missing values detected.", "\n")

# Summary for categorical variables
print("\nFrequency Distribution for 'group' (Categorical Variable):")
print(plant_growth['group'].value_counts(), "\n")

print("\nDataset Description:")
print(f"""
The PlantGrowth dataset consists of {len(plant_growth)} observations and {plant_growth.shape[1]} columns:
1. 'weight': A numerical variable representing the weight of plants.
   - Range: {plant_growth['weight'].min()} to {plant_growth['weight'].max()} grams
   - Mean: {plant_growth['weight'].mean():.3f} grams
   - Standard Deviation: {plant_growth['weight'].std():.3f} grams
2. 'group': A categorical variable with three treatment groups:
   - Control group ('ctrl'): {plant_growth['group'].value_counts()['ctrl']} observations
   - Treatment 1 ('trt1'): {plant_growth['group'].value_counts()['trt1']} observations
   - Treatment 2 ('trt2'): {plant_growth['group'].value_counts()['trt2']} observations
""")


Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   rownames  30 non-null     int64  
 1   weight    30 non-null     float64
 2   group     30 non-null     object 
dtypes: float64(1), int64(1), object(1)
memory usage: 848.0+ bytes

Missing Values in Dataset:
No missing values detected. 


Frequency Distribution for 'group' (Categorical Variable):
group
ctrl    10
trt1    10
trt2    10
Name: count, dtype: int64 


Dataset Description:

The PlantGrowth dataset consists of 30 observations and 3 columns:
1. 'weight': A numerical variable representing the weight of plants.
   - Range: 3.59 to 6.31 grams
   - Mean: 5.073 grams
   - Standard Deviation: 0.701 grams
2. 'group': A categorical variable with three treatment groups:
   - Control group ('ctrl'): 10 observations
   - Treatment 1 ('trt1'): 10 observations
   - Treatment 2 ('trt2'): 